In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/efficientnet-model/efficientnetb0_notop.h5


In [2]:
# Install required libraries
!pip install tensorflow kagglehub

# Import libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt


In [3]:
# Data augmentation layers
random_rotation = tf.keras.layers.RandomRotation(0.2, dtype=tf.float32)
random_zoom = tf.keras.layers.RandomZoom(0.2, dtype=tf.float32)
random_flip = tf.keras.layers.RandomFlip("horizontal", dtype=tf.float32)
random_contrast = tf.keras.layers.RandomContrast(0.3, dtype=tf.float32)

def apply_augmentation(img):
    choice = tf.random.uniform(shape=[], minval=0, maxval=7, dtype=tf.int32)
    
    augmentations = [
        lambda: random_flip(img),
        lambda: tf.image.random_brightness(img, 0.4),
        lambda: random_contrast(img),
        lambda: random_rotation(tf.expand_dims(img, 0))[0],
        lambda: random_zoom(tf.expand_dims(img, 0))[0],
        lambda: tf.image.random_hue(img, 0.3),
        lambda: tf.image.random_saturation(img, 0.6, 1.6)
    ]
    
    return tf.switch_case(choice, augmentations)

def augment_image(image, label):
    image = apply_augmentation(image)
    return preprocess_input(image), label

In [5]:
def create_dataset(directory, augment=False):
    ds = tf.keras.utils.image_dataset_from_directory(
        directory,
        image_size=(256, 256),
        batch_size=None,
        shuffle=augment,
        seed=42 if augment else None
    )
    
    # Apply options to handle corrupt images and improve performance
    options = tf.data.Options()
    options.experimental_deterministic = False  # Enable non-deterministic shuffling
    ds = ds.with_options(options)
    ds = ds.apply(tf.data.experimental.ignore_errors())  # Ignore corrupted images
    
    if augment:
        ds = ds.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    
    return ds.batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Download dataset
import kagglehub
dataset_path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")
base_dir = os.path.join(dataset_path, "Dataset")

# Create datasets
train_ds = create_dataset(os.path.join(base_dir, "Train"), augment=True)
val_ds = create_dataset(os.path.join(base_dir, "Validation"))
test_ds = create_dataset(os.path.join(base_dir, "Test"))

In [11]:
# Path to the uploaded weights file
weights_path = '/kaggle/input/efficientnet-model/efficientnetb0_notop.h5'

# Load the model with the downloaded weights
base_model = EfficientNetB0(weights=weights_path, include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # Freeze the base model initially

# Add custom layers on top of the base model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid', dtype='float32')(x)  # Changed dtype to float32

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=output)

# Print model summary to verify
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_4 (Rescaling)   │ (None, 256, 256, 3)    │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_4           │ (None, 256, 256, 3)    │              7 │ rescaling_4[0][0]      │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 257, 257, 3)    │              0 │ normalization_4[0][0]  │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 128, 128, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 128, 128, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 128, 128, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 128, 128, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 128, 128, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 128, 128, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand         │ (None, 1, 1, 32)       │            288 │ block1a_se_reduce[0][… │
│ (Conv2D)             

 Total params: 4,708,004 (17.96 MB)

 Trainable params: 657,409 (2.51 MB)

 Non-trainable params: 4,050,595 (15.45 MB)

In [12]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_auc', mode='max', factor=0.5, patience=2, min_lr=1e-7, verbose=1
)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "best_model.keras", monitor='val_auc', mode='max', save_best_only=True, verbose=1
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', mode='max', patience=5, restore_best_weights=True, verbose=1
)


In [13]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, amsgrad=True),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Train the model (Phase 1 - Frozen Training)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,  # Number of epochs for Phase 1
    callbacks=[reduce_lr, checkpoint_cb, earlystop_cb]
)


Epoch 1/20
   2188/Unknown 909s 402ms/step - accuracy: 0.7405 - auc: 0.8202 - loss: 4.1266

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_auc improved from -inf to 0.91580, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1050s 467ms/step - accuracy: 0.7405 - auc: 0.8202 - loss: 4.1258 - val_accuracy: 0.8130 - val_auc: 0.9158 - val_loss: 0.9091 - learning_rate: 1.0000e-04
Epoch 2/20
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.8020 - auc: 0.8867 - loss: 0.7943
Epoch 2: val_auc did not improve from 0.91580
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 433s 198ms/step - accuracy: 0.8020 - auc: 0.8867 - loss: 0.7943 - val_accuracy: 0.8007 - val_auc: 0.9135 - val_loss: 0.5828 - learning_rate: 1.0000e-04
Epoch 3/20
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.8083 - auc: 0.8943 - loss: 0.5423
Epoch 3: val_auc improved from 0.91580 to 0.91972, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 425s 194ms/step - accuracy: 0.8083 - auc: 0.8943 - loss: 0.5423 - val_accuracy: 0.8353 - val_auc: 0.9197 - val_loss: 0.4490 - learning_rate: 1.0000e-04
Epoch 4/20
2188/2188 ━━━━━━━━━

In [16]:
# 1. Unfreeze last 20 layers
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False  # Freeze all except last 20 layers

# 2. Recompile with lower LR
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Smaller LR for fine-tuning
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# 3. Define callbacks (critical!)
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_auc', mode='max', factor=0.5, patience=2, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "best_finetuned_model.keras",  # New name to avoid conflicts
        monitor='val_auc', save_best_only=True, mode='max', verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_auc', patience=5, restore_best_weights=True, mode='max', verbose=1
    )
]

In [17]:
# 4. Execute fine-tuning
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks  # Uses the callbacks defined in Cell 1
)

# 5. Save final model (in another cell)
model.save("final_finetuned_model.keras")
print("Models saved:")
print("- best_finetuned_model.keras (best validation performance)")
print("- final_finetuned_model.keras (final trained state)")

Epoch 1/10
   2188/Unknown 561s 245ms/step - accuracy: 0.8126 - auc: 0.8965 - loss: 0.4310
Epoch 1: val_auc improved from -inf to 0.94647, saving model to best_finetuned_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 625s 274ms/step - accuracy: 0.8126 - auc: 0.8965 - loss: 0.4309 - val_accuracy: 0.8670 - val_auc: 0.9465 - val_loss: 0.3314 - learning_rate: 1.0000e-05
Epoch 2/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.8772 - auc: 0.9510 - loss: 0.3073
Epoch 2: val_auc improved from 0.94647 to 0.95881, saving model to best_finetuned_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 423s 193ms/step - accuracy: 0.8772 - auc: 0.9511 - loss: 0.3073 - val_accuracy: 0.8854 - val_auc: 0.9588 - val_loss: 0.2936 - learning_rate: 1.0000e-05
Epoch 3/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.8958 - auc: 0.9638 - loss: 0.2673
Epoch 3: val_auc improved from 0.95881 to 0.96594, saving model to best_finetuned_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 478s 218ms/step - accurac